In [1]:
import torch 
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from PIL import Image
from tqdm import tqdm

# 같은 돌고래 혹은 고래가 여러장 찍힌 사진이 들어있다.

# Create Dataloader

In [2]:
class Whale_Dolphin_Dataset(Dataset) :
    def __init__(self, file_path_list, label_list, classes_list, dataset_root): 
        self.file_path_list = file_path_list
        self.label_list = label_list
        self.classes_list = classes_list
        self.dataset_root = dataset_root
    
    # return the length of file_path_list
    def __len__(self): 
        return len(self.file_path_list) 
    
    def __getitem__(self, index): 
        
        # image processing
        to_tensor = transforms.ToTensor()
        img = to_tensor(Image.open(self.dataset_root + self.file_path_list[index]).convert('RGB'))
        img.resize_(3, 224, 224)
        img = torch.divide(img, 255.0) # 텐서로 변경 후 이미지 리사이징하고 각 채널을 0~1 사이의 값으로 만들어버림
        
        label = self.classes_list.index(self.label_list[index])
        
        return img, label


In [3]:
def make_dataloader(dataset_root) : 
    train_label_csv = pd.read_csv(dataset_root + 'train.csv')
    train_image_file_list = train_label_csv['image'].tolist()
    train_image_label_list = train_label_csv['species'].tolist()
    classes_list = list(set(train_image_label_list))
    
    train_dataset = Whale_Dolphin_Dataset(train_image_file_list, train_image_label_list, classes_list, dataset_root + 'train_images/')
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    return train_dataloader

In [4]:
dataset_root = '/home/minkyukim/Dataset/happy-whale-and-dolphin/'
train_dataloader = make_dataloader(dataset_root)

# Make model